# **Κεφάλαιο 1ο: Dataset**
---
To dataset πάνω στο οποίο θα βασίζεται η παρακάτω εργασία είναι:

**Stellar Classification Dataset - SDSS17**

Classification of Stars, Galaxies and Quasars.

**Sloan Digital Sky Survey DR17**

**Link**: https://www.kaggle.com/datasets/fedesoriano/stellar-classification-dataset-sdss17

## **1.1 Περιβάλλον**
---
Στην αστρονομία, μπορεί να γίνει ταξινόμηση των αστέρων με βάση τα φασματικά τους χαρακτηριστικά. Το σύστημα ταξινόμησης των γαλαξιών, των κβάζαρ και των αστέρων είναι ένα από τα πιο θεμελιώδη στην αστρονομία.

Αυτό το dataset αποσκοπεί στην ταξινόμηση αστέρων, γαλαξιών και κβάζαρ με βάση τα φασματικά χαρακτηριστικά τους.

## **1.2 Περιεχόμενο**
---
Τα δεδομένα αποτελούνται από 100.000 παρατηρήσεις του διαστήματος που έγιναν από τη SDSS (Sloan Digital Sky Survey). Κάθε παρατήρηση περιγράφεται από 17 στήλες χαρακτηριστικών και 1 στήλη κλάσης που την προσδιορίζει είτε ως αστέρι, είτε ως γαλαξία, είτε ως κβάζαρ.

Περισσότερα για την περιγραφή του κάθε feature εδώ: https://www.kaggle.com/datasets/fedesoriano/stellar-classification-dataset-sdss17

# **Κεφάλαιο 2ο: Preprocessing**
---
Ακολουθεί μια σειρά από ενέργειες προ επεξεργασίας έτσι ώστε να καθαρίσει το dataset και να βοηθήσουμε την εκπαίδευση του μοντέλου.

## **2.1 Load the needed libraries**
---
Φορτώνουμε τις απαραίτητες βιβλιοθήκες.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

## **2.2 Read the dataset**
---
Φτιάχνουμε το path προς το dataset και διαβάζουμε το dataset.


In [3]:
# Initialize the dataset path
dataset_path = '/kaggle/input/stellar-classification-dataset-sdss17/star_classification.csv'

# Read the Dataset
stars = pd.read_csv(dataset_path)

## **2.3 Drop any unnecessary columns**
---

### 2.3.1 Drop 'obj_ID' and 'spec_obj_ID' columns that refer to unique IDs for each observation.
---
Παρατηρήθηκε πως στο dataset υπάρχουν columns τα οποία περιέχουν μοναδικά αναγνωριστικά για κάθε παρατήρηση.
Αυτά τα features δεν παρέχουν καμία σημαντική πληροφορία για την πρόβλεψη του target class καθώς
η ύπαρξη άσχετων (προς τη μάθηση) χαρακτηριστικών μπορεί να αυξήσουν την πολυπλοκότητα του μοντέλου χωρίς να βοηθάνε το μοντέλο στο να μάθει.
Συνεπώς, επιλέχθηκε να αφαιρεθούν.

In [4]:
# Drop unique columns aka IDs
processed = stars.drop(columns=['obj_ID', 'spec_obj_ID'])
print(processed.columns)

Index(['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'run_ID', 'rerun_ID',
       'cam_col', 'field_ID', 'class', 'redshift', 'plate', 'MJD', 'fiber_ID'],
      dtype='object')


### 2.3.2 Drop columns with zero deviation
---
Παράλληλα, παρατηρήθηκε πως υπάρχει feature που για κάθε αστρική παρατήρηση έχει την ίδια τιμή (μηδενική απόκλιση).
Αυτό το feature δεν παρέχει καμία περαιτέρω πληροφορία στη μάθηση και επιλέγεται να αφαιρεθεί.

In [4]:
# Check for columns with zero deviation
print(processed.nunique())

alpha       99999
delta       99999
u           93748
g           92651
r           91901
i           92019
z           92007
run_ID        430
rerun_ID        1
cam_col         6
field_ID      856
class           3
redshift    99295
plate        6284
MJD          2180
fiber_ID     1000
dtype: int64


In [5]:
# Drop these columns
processed = processed.drop(columns=['rerun_ID'])
print(processed.columns)

Index(['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'run_ID', 'cam_col',
       'field_ID', 'class', 'redshift', 'plate', 'MJD', 'fiber_ID'],
      dtype='object')


### 2.3.3 Check for null values
---
Έπειτα γίνεται έλεγχος για rows που έχουν null τιμές ώστε να αφαιρεθούν ή να γίνει imputation.
Παρατηρήθηκε πως δεν υπάρχουν rows με null τιμές.

In [6]:
if processed.isnull().values.any():
    print(processed.isnull().sum())
else:
    print('No missing values')

No missing values


### 2.3.4 Check for imbalances in the distribution of classes
---
Γίνεται έλεγχος για την κατανομή των κλάσεων μέσα στο dataset. Γίνεται δηλαδή έλεγχος για το αν κάποια κλάση υπερέχει τις υπόλοιπες ή κάποια κλάση έχει πολύ μικρό ποσοστό των samples του dataset.

In [6]:
class_distribution = stars['class'].value_counts()  # Get class distribution
print(class_distribution)   # Print the distribution

class
GALAXY    59445
STAR      21594
QSO       18961
Name: count, dtype: int64


Παρατηρούμε πως οι 3 κλάσεις δεν είναι ισοκατανεμημένες στο dataset.

Θα εκτελεστεί στη συνέχεια στο training set κάποιος αλγόριθμος oversampling.

## **2.4 Label Encode the target class**
---
Η κλάση πρόβλεψης αποτελείται από 3 κλάσεις (GALAXY, STAR ή QSO) και ο τύπος τιμών που παίρνει είναι String.
Συνεπώς, εκτελείται Label Encoding από τη βιβλιοθήκη της sklearn έτσι ώστε οι τιμές GALAXY, STAR και QSO να αντιστοιχηθούν σε κάποιον ακέραιο αριθμό (0, 1, 2).

Αρχικά εκτελείται Label Encoding για τη χρήση του στις κλάσεις του Nearest Centroid και k-NN, αφού δεν υποστηρίζουν το One-Hot Encoding.

In [7]:
target_encoder = LabelEncoder() # Initialize the constructor
processed['class'] = target_encoder.fit_transform(processed['class'])   # Label Encode the target class
print(processed['class'].head(10))  # Print the first 10 rows

0    0
1    0
2    0
3    0
4    0
5    1
6    1
7    0
8    0
9    2
Name: class, dtype: int64


## **2.5 Using One Hot Encoding for cam_col feature column**
---
Το feature cam_col μπορεί να πάρει από ένα εύρος 6 διακριτών τιμών.
Συνεπώς, πρόκειται για μια κατηγορική μεταβλητή όπου ο αριθμός των κατηγοριών δεν είναι ταξινομημένος (δηλαδή δεν υπάρχει εγγενής τάξη μεταξύ των κατηγοριών).
Άρα, γίνεται χρήση του OneHot Encoding από τη βιβλιοθήκη της sklearn για τη μετατροπή του feature σε 6 columns που αποτελούν τη δυαδική αναπαράσταση των τιμών της κατηγορικής μεταβλητής.

In [8]:
cam_col_encoder = OneHotEncoder(sparse_output=False)    # Initialize the OneHotEncoder constructor
encoded = cam_col_encoder.fit_transform(processed[['cam_col']]) # One Hot encode the cam_col feature
encoded_df = pd.DataFrame(encoded, columns=cam_col_encoder.get_feature_names_out(['cam_col']))  # Wrap it into a DataFrame
processed = pd.concat([processed, encoded_df], axis=1)  # Concat the different columns
processed = processed.drop(['cam_col'], axis=1) # Drop the initial feature
print(processed.columns)    # Print the updated features of the dataset

Index(['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'run_ID', 'field_ID',
       'class', 'redshift', 'plate', 'MJD', 'fiber_ID', 'cam_col_1',
       'cam_col_2', 'cam_col_3', 'cam_col_4', 'cam_col_5', 'cam_col_6'],
      dtype='object')


## **2.6 Split the dataset to train and test**
---
Μετά την προεπεξεργασία του dataset γίνεται χρήση της train_test_split από τη βιβλιοθήκη της sklearn για τον χωρισμό του dataset σε δεδομένα εκπαίδευσης και δοκιμής κατά αναλογία 60-40.

In [9]:
X = processed.drop(columns=['class'])  # Drop the target prediction class
Y = processed['class']  # Store the target class
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4)    # Train Test Split

## **2.7 SMOTE**
---
Θα γίνει ένα oversample στις minority classes έτσι ώστε να υπάρχει μια ισορροπία μεταξύ των κλάσεων στο training.

Το SMOTE λειτουργεί με τη δημιουργία συνθετικών δειγμάτων για τις κλάσεις με μικρό ποσοστό εμφάνισης. Αντί να βασίζεται αποκλειστικά στα αρχικά δείγματα των μειονοτικών κλάσεων, το SMOTE δημιουργεί συνθετικά samples με παρεμβολή μεταξύ των υπαρχόντων δειγμάτων.

Επιλέχτηκε γιατί κατά γενική ομολογία παράγει καλύτερα αποτελέσματα από κάποιον άλλον απλό αλγόριθμο ισορρόπησης dataset, όπως ένα Random Oversampling.

In [10]:
from imblearn.over_sampling import SMOTE
smt = SMOTE(random_state=42)    # Initialize the SMOTE constructor
X_train, Y_train = smt.fit_resample(X_train, Y_train)   # SMOTE the training set
class_distribution = Y_train.value_counts() # Get the updated distribution of the classes
print(class_distribution)   # Print the updated distribution

class
2    35708
0    35708
1    35708
Name: count, dtype: int64


Παρατηρούμε ότι μετά την εκτέλεση του SMOTE κάθε κλάση έχει ακριβώς το ίδιο πλήθος δειγμάτων.

## **2.8 Normalize features**
---
Γίνεται κανονικοποίηση των features έτσι ώστε να βελτιώσουμε την απόδοση και τη σταθερότητα των μοντέλων.

Αποτρέπουμε τα χαρακτηριστικά με μεγαλύτερες κλίμακες από το να κυριαρχούν σε εκείνα με μικρότερες κλίμακες κατά τη διάρκεια του training.

In [11]:
scaler = StandardScaler()   # Initialize StandardScaler
X_train = pd.DataFrame(scaler.fit_transform(X_train))   # Normalize the training set of features and convert the numpy.array to pandas.DataFrame
X_test = pd.DataFrame(scaler.transform(X_test)) # Normalize the test set of features and convert the numpy.array to pandas.DataFrame

# **Κεφάλαιο 3ο: Support Vector Machines**
---
Γίνεται εισαγωγή των κατάλληλων βιβλιοθηκών για την υλοποίηση και έρευνα των SVMs.

In [12]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from time import time
from sklearn.metrics import classification_report

## **3.1 Linear SVMs**
---
Στο κομμάτι αυτό γίνεται δοκιμή διαφόρων τιμών για την υπερπαράμετρο C.
Χρησιμοποιείται η βιβλιοθήκη SVC της sklearn με kernel=linear.
Για την δοκιμή εκπαιδεύονται μοντέλα στα τελευταία 10.000 δείγματα του training set, δηλαδή το 10% του.
Παρόλο που χρησιμοποιείται ένα μικρό μέρος του dataset παρατηρήθηκε ότι επιτυγχάνεται παρόμοια απόδοση σε ολόκληρο το validation set καθώς το υποσύνολό του καταγράφει αρκετή ποικιλομορφία για την αποτελεσματική εκπαίδευση του μοντέλου.

In [27]:
models = []
for C in [0.1, 1, 10, 100, 1000, 5000]:
    model = SVC(C=C, kernel='linear')
    start = time()
    model.fit(X_train[:10000], Y_train[:10000])
    end = time()
    models.append(model)
    print("Number of support vectors = ", model.n_support_)
    print("Fit time: ", end-start)
    print("C = ", C, "Train/Test acc = ", model.score(X_train[:10000], Y_train[:10000]), model.score(X_test, Y_test))

Number of support vectors =  [1421  416 1132]
Fit time:  1.0631299018859863
C =  0.1 Train/Test acc =  0.946 0.9418
Number of support vectors =  [839 368 538]
Fit time:  1.2939202785491943
C =  1 Train/Test acc =  0.9558 0.951375
Number of support vectors =  [533 359 228]
Fit time:  4.114961862564087
C =  10 Train/Test acc =  0.9651 0.961125
Number of support vectors =  [452 358 127]
Fit time:  15.494871139526367
C =  100 Train/Test acc =  0.9662 0.9633
Number of support vectors =  [430 361 100]
Fit time:  87.41770029067993
C =  1000 Train/Test acc =  0.9671 0.963625
Number of support vectors =  [422 359  95]
Fit time:  406.75839471817017
C =  5000 Train/Test acc =  0.9677 0.964025


**Παρατηρούμε ότι όσο αυξάνουμε το C παίρνουμε καλύτερες αποδόσεις στο training/test set.
Όσο περισσότερο αυξάνουμε το C τόσο μικρότερες είναι και οι βελτιώσεις.
Παράλληλα, αυξάνεται δραματικά και ο χρόνος εκπαίδευσης.**

In [28]:
finest_model_linear = models[5]
Y_pred = finest_model_linear.predict(X_test)
print("Train/Test micro accuracy = ", finest_model_linear.score(X_train[:10000], Y_train[:10000]), finest_model_linear.score(X_test, Y_test))
# Generate the classification report
report = classification_report(Y_test, Y_pred)
# Print the report
print(report)

Train/Test micro accuracy =  0.9677 0.964025
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     23737
           1       0.95      0.89      0.92      7591
           2       0.98      1.00      0.99      8672

    accuracy                           0.96     40000
   macro avg       0.96      0.95      0.96     40000
weighted avg       0.96      0.96      0.96     40000



**Σχολιασμός**
---
**Ο γραμμικός SVM επιτυγχάνει την μεγαλύτερη ακρίβεια γύρω στο 96% με C=5000.**

## **3.2 Polynomial SVMs**
---
Στο κομμάτι αυτό γίνεται δοκιμή διαφόρων τιμών για την επίδραση του βαθμού του πολυωνύμου και της υπερπαράμετρου gamma.
Χρησιμοποίειται η βιβλιοθήκη SVC της sklearn με kernel=poly.
Χρησιμοποιείται και πάλι ένα 10% του dataset.

In [21]:
# Exploration of degree parameter for polynomial kernel
for degree in [2,3,4,5]:
    model = SVC(C=1000, kernel='poly', degree=degree)
    start = time()
    model.fit(X_train[:10000], Y_train[:10000])
    end = time()
    print("Number of support vectors = ", model.n_support_)
    print("Fit time: ", end-start)
    print("Degree = ", degree, "Train/Test acc = ", model.score(X_train[:10000], Y_train[:10000]), model.score(X_test, Y_test))

Number of support vectors =  [1080  984 1089]
Fit time:  92.22521948814392
Degree =  2 Train/Test acc =  0.9084 0.901175
Number of support vectors =  [649 335 483]
Fit time:  44.59967255592346
Degree =  3 Train/Test acc =  0.9837 0.9462
Number of support vectors =  [977 463 751]
Fit time:  12.780603408813477
Degree =  4 Train/Test acc =  0.9984 0.90715
Number of support vectors =  [1162  435  848]
Fit time:  6.511460542678833
Degree =  5 Train/Test acc =  0.9997 0.902725


**Παρατηρούμε ότι αυξάνοντας το βαθμό του πολυώνυμου στην συνάρτηση πυρήνα το μοντέλο μαθαίνει καλύτερα τα δεδομένα εκπαίδευσης. 
Ταυτόχρονα όμως μειώνεται η ακρίβεια στο validation set.
Το μοντέλο πιθανότητα υποφέρει από overfitting.**

**Το μοντέλο με βαθμό 3 πετυχαίνει ακρίβεια στο test set: 93%**

In [19]:
# Exploration of gamma parameter for polynomial kernel
for gamma in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    model = SVC(C=1000, kernel='poly', degree=3, gamma=gamma, cache_size=5000)
    start = time()
    model.fit(X_train[:10000], Y_train[:10000])
    end = time()
    print("Number of support vectors = ", model.n_support_)
    print("Fit time: ", end-start)
    print("Gamma = ", gamma, "Train/Test acc = ", model.score(X_train[:10000], Y_train[:10000]), model.score(X_test, Y_test))

Number of support vectors =  [644 309 525]
Fit time:  58.34542179107666
Gamma =  0.1 Train/Test acc =  0.9987 0.9251
Number of support vectors =  [631 293 511]
Fit time:  76.38606357574463
Gamma =  0.2 Train/Test acc =  0.9999 0.9225
Number of support vectors =  [631 293 511]
Fit time:  65.45203995704651
Gamma =  0.3 Train/Test acc =  1.0 0.921725
Number of support vectors =  [630 293 511]
Fit time:  76.94806265830994
Gamma =  0.4 Train/Test acc =  1.0 0.92165
Number of support vectors =  [632 293 511]
Fit time:  61.83602213859558
Gamma =  0.5 Train/Test acc =  1.0 0.9217
Number of support vectors =  [631 293 511]
Fit time:  65.71755313873291
Gamma =  0.6 Train/Test acc =  1.0 0.921725
Number of support vectors =  [632 293 511]
Fit time:  75.02367758750916
Gamma =  0.7 Train/Test acc =  1.0 0.92165
Number of support vectors =  [630 293 511]
Fit time:  77.08167099952698
Gamma =  0.8 Train/Test acc =  1.0 0.92165
Number of support vectors =  [632 293 511]
Fit time:  70.35230350494385
Gam

**H παράμετρος gamma καθορίζει πόσο μακριά φτάνει η επιρροή ενός μεμονωμένου training sample. Οι χαμηλές τιμές να σημαίνουν «μακριά» και οι υψηλές που να σημαίνουν «κοντά».**
**Παρατηρούμε πως με την αύξηση της υπερπαραμέτρου gamma το μοντέλο μαθαίνει τέλεια (Ακρίβεια 100%) τα δεδομένα εκπαίδευσης. Παράλληλα όμως δεν καταφέρνει να επιτύχει μεγαλύτερη ακρίβεια στα δεδομένα δοκιμής.**

## **3.2.1 Εύρεση Υπερπαραμέτρων**
Γίνεται εκτέλεση ενός Grid Search για την εύρεση βέλτιστων υπερπαραμέτρων με σκοπό την επίτευξη της μεγαλύτερης δυνατής ακρίβειας.
Χρησιμοποιείται ένα 2% του dataset (2000 δείγματα) λόγω της εκθετικής πολυπλοκότητας του Grid Search.
Αναζητούμε έναν βέλτιστο συνδυασμό στα 2000 δείγματα και ελέγχουμε αν αυτός ο συνδυασμός μπορεί να τα πάει καλά σε ολόκληρο το validation set και ουσιαστικά να γενικεύσει επιτυχώς.

In [19]:
# Parameter Grid to explore hyperparameters
parameters = {
    'C': [0.1, 1, 10, 100, 1000],
    'degree': [2,3,4,5],
    'gamma': ['scale','auto', 0.1, 0.5, 0.9]
}

# Create a polynomial SVM model
poly_svm = SVC(kernel='poly', cache_size=5000)

# Create a GridSearchCV object
grid_search = GridSearchCV(poly_svm, parameters, cv=3, scoring='accuracy', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train[:2000], Y_train[:2000])

# Print the best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Get the best model
best_model = grid_search.best_estimator_

Best Parameters: {'C': 10, 'degree': 3, 'gamma': 'scale'}
Accuracy on Test Set: 0.89395


In [25]:
# Train the best model to a bigger percentage of the dataset and check if the performance increased
finest_model_poly = best_model
Y_pred = finest_model_poly.predict(X_test)
print("Train/Test micro accuracy = ", finest_model_poly.score(X_train[:10000], Y_train[:10000]), finest_model_poly.score(X_test, Y_test))
# Generate the classification report
report = classification_report(Y_test, Y_pred)
# Print the report
print(report)

Train/Test micro accuracy =  0.9642 0.943675
              precision    recall  f1-score   support

           0       0.95      0.96      0.95     23737
           1       0.95      0.90      0.92      7591
           2       0.92      0.95      0.93      8672

    accuracy                           0.94     40000
   macro avg       0.94      0.93      0.94     40000
weighted avg       0.94      0.94      0.94     40000



**Σχολιασμός**
---
**Παρατηρούμε πως το καλύτερο πολυωνυμικό μοντέλο επιτυγχάνει μια ακρίβεια 94%.**

## **3.3 RBF SVMs**
---
Στο κομμάτι αυτό γίνεται δοκιμή διαφόρων τιμών για την επίδραση της υπερπαράμετρου gamma στα SVM με απείρου βαθμού kernel.
Χρησιμοποίειται η βιβλιοθήκη SVC της sklearn με kernel=rbf.
Χρησιμοποιείται και πάλι ένα 10% του dataset.

In [22]:
# Exploration of gamma parameter for rbf kernel
for gamma in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    model = SVC(C=1000, kernel='rbf', gamma=gamma, cache_size=5000)
    start = time()
    model.fit(X_train[:10000], Y_train[:10000])
    end = time()
    print("Number of support vectors = ", model.n_support_)
    print("Fit time: ", end-start)
    print("Gamma = ", gamma, "Train/Test acc = ", model.score(X_train[:10000], Y_train[:10000]), model.score(X_test, Y_test))

Number of support vectors =  [1171  466  937]
Fit time:  2.419848680496216
Gamma =  0.1 Train/Test acc =  1.0 0.9257
Number of support vectors =  [2003  761 1453]
Fit time:  3.5820634365081787
Gamma =  0.2 Train/Test acc =  1.0 0.909475
Number of support vectors =  [2689 1065 1727]
Fit time:  4.858321189880371
Gamma =  0.3 Train/Test acc =  1.0 0.896425
Number of support vectors =  [3205 1292 1886]
Fit time:  6.650872468948364
Gamma =  0.4 Train/Test acc =  1.0 0.88425
Number of support vectors =  [3619 1451 1972]
Fit time:  8.910874366760254
Gamma =  0.5 Train/Test acc =  1.0 0.872225
Number of support vectors =  [3940 1561 2031]
Fit time:  9.398403406143188
Gamma =  0.6 Train/Test acc =  1.0 0.859125
Number of support vectors =  [4254 1655 2074]
Fit time:  10.016894817352295
Gamma =  0.7 Train/Test acc =  1.0 0.84455
Number of support vectors =  [4516 1723 2098]
Fit time:  9.240455627441406
Gamma =  0.8 Train/Test acc =  1.0 0.828125
Number of support vectors =  [4724 1772 2115]
Fit 

**Παρατηρούμε και πάλι πως με την αύξηση της υπερπαραμέτρου gamma το μοντέλο μαθαίνει τέλεια (Ακρίβεια 100%) τα δεδομένα εκπαίδευσης. Παράλληλα όμως δεν καταφέρνει να επιτύχει μεγαλύτερη ακρίβεια στα δεδομένα δοκιμής. Όσο αυξάνουμε το gamma το μοντέλο όλο και περισσότερο υποφέρει από overfitting.**

## **3.3.1 Εύρεση Υπερπαραμέτρων**
Γίνεται εκτέλεση ενός Grid Search για να βρούμε τις βέλτιστες υπερπαραμέτρους για τον rbf kernel με στόχο την επίτευξη της μεγαλύτερης δυνατής απόδοσης.

In [25]:
# Parameter Grid to explore hyperparameters
parameters = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': ['scale', 'auto', 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
}

# Create a Radial Basis Function SVM model
rbf_svm = SVC(kernel='rbf')

# Create a GridSearchCV object
grid_search = GridSearchCV(rbf_svm, parameters, cv=3, scoring='accuracy', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train[:10000], Y_train[:10000])

# Print the best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Get the best model
best_model = grid_search.best_estimator_

Best Parameters: {'C': 10, 'gamma': 'auto'}


In [24]:
finest_model_rbf = best_model
Y_pred = finest_model_rbf.predict(X_test)
print("Train/Test micro accuracy = ", finest_model_rbf.score(X_train[:10000], Y_train[:10000]), finest_model_rbf.score(X_test, Y_test))
# Generate the classification report
report = classification_report(Y_test, Y_pred)
# Print the report
print(report)

Train/Test micro accuracy =  0.9642 0.943675
              precision    recall  f1-score   support

           0       0.95      0.96      0.95     23737
           1       0.95      0.90      0.92      7591
           2       0.92      0.95      0.93      8672

    accuracy                           0.94     40000
   macro avg       0.94      0.93      0.94     40000
weighted avg       0.94      0.94      0.94     40000



**Σχολιασμός**
---
**Παρατηρούμε ότι το SVM με πυρήνα RBF επιτυγχάνει μια ακρίβεια 94% στο validation set.**

## **3.4 Sigmoid SVMs**
---
Στο κομμάτι αυτό γίνεται δοκιμή διαφόρων τιμών για την επίδραση της υπερπαράμετρου gamma στα SVM με σιγμοειδή συνάρτηση πυρήνα.
Χρησιμοποίειται η βιβλιοθήκη SVC της sklearn με kernel=sigmoid.
Χρησιμοποιείται και πάλι ένα 10% του dataset.

In [24]:
# Exploration of gamma parameter for sigmoid kernel
for gamma in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    model = SVC(C=100, kernel='sigmoid', gamma=gamma, cache_size=5000)
    start = time()
    model.fit(X_train[:10000], Y_train[:10000])
    end = time()
    print("Number of support vectors = ", model.n_support_)
    print("Fit time: ", end-start)
    print("Gamma = ", gamma, "Train/Test accuracy = ", model.score(X_train[:10000], Y_train[:10000]), model.score(X_test, Y_test))

Number of support vectors =  [1981  670 1605]
Fit time:  3.6933846473693848
Gamma =  0.1 Train/Test acc =  0.6723 0.68075
Number of support vectors =  [2258  936 1717]
Fit time:  5.060818672180176
Gamma =  0.2 Train/Test acc =  0.6119 0.62365
Number of support vectors =  [2297 1007 1729]
Fit time:  5.466294765472412
Gamma =  0.3 Train/Test acc =  0.5934 0.604675
Number of support vectors =  [1955  908 1729]
Fit time:  5.586326599121094
Gamma =  0.4 Train/Test acc =  0.5739 0.5812
Number of support vectors =  [2102 1053 1729]
Fit time:  6.1299333572387695
Gamma =  0.5 Train/Test acc =  0.5884 0.59285
Number of support vectors =  [2262 1086 1731]
Fit time:  6.045241594314575
Gamma =  0.6 Train/Test acc =  0.5817 0.589025
Number of support vectors =  [2421  982 1551]
Fit time:  5.444751262664795
Gamma =  0.7 Train/Test acc =  0.5414 0.54445
Number of support vectors =  [2281  991 1674]
Fit time:  5.472646474838257
Gamma =  0.8 Train/Test acc =  0.5572 0.56075
Number of support vectors =  

## **3.4.1 Εύρεση Υπερπαραμέτρων**
Γίνεται εκτέλεση ενός Grid Search για να βρούμε τις βέλτιστες υπερπαραμέτρους για τον σιγμοειδή πυρήνα με στόχο την επίτευξη της μεγαλύτερης δυνατής απόδοσης.

In [33]:
# Parameter Grid to explore hyperparameters
parameters = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': ['scale', 'auto', 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
}

# Create a Radial Basis Function SVM model
sigmoid_svm = SVC(kernel='rbf')

# Create a GridSearchCV object
grid_search = GridSearchCV(sigmoid_svm, parameters, cv=3, scoring='accuracy', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train[:2000], Y_train[:2000])

# Print the best parameters
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Get the best model
best_model = grid_search.best_estimator_

Best Parameters: {'C': 10, 'gamma': 'scale'}


In [32]:
finest_model_sigmoid = best_model
Y_pred = finest_model_sigmoid.predict(X_test)
print("Train/Test micro accuracy = ", finest_model_sigmoid.score(X_train[:10000], Y_train[:10000]), finest_model_sigmoid.score(X_test, Y_test))
# Generate the classification report
report = classification_report(Y_test, Y_pred)
# Print the report
print(report)

Train/Test micro accuracy =  0.9108 0.9023
              precision    recall  f1-score   support

           0       0.90      0.94      0.92     23737
           1       0.97      0.84      0.90      7591
           2       0.86      0.84      0.85      8672

    accuracy                           0.90     40000
   macro avg       0.91      0.88      0.89     40000
weighted avg       0.90      0.90      0.90     40000



**Σχολιασμός**
---
**Παρατηρούμε ότι το SVM με πυρήνα σιγμοειδή επιτυγχάνει μια ακρίβεια 90% στο validation set.**

# **Κεφάλαιο 4: Συγκρίσεις**

## **4.0 Null accuracy**
---
Η μηδενική ακρίβεια είναι η ακρίβεια που θα μπορούσε να επιτευχθεί με την πρόβλεψη πάντα της πιο συχνής κλάσης.

Πρέπει να συγκρίνουμε τις αποδόσεις των μοντέλων μας με τη μηδενική ακρίβεια για να αποφανθούμε αν τα μοντέλα μας έχουν καλή απόδοση.

In [54]:
# Get the most frequent class
Y_test.value_counts()

class
0    23751
2     8640
1     7609
Name: count, dtype: int64

In [55]:
# Calculate the null accuracy
null_accuracy = (23751/(23751+8640+7609))
# Print the null accuracy
print('Null accuracy score: {0:0.4f}'. format(null_accuracy))

Null accuracy score: 0.5938


**Παρατηρούμε πως όλα τα μοντέλα SVM που παρουσιάστηκαν επιτυγχάνουν μεγαλύτερη ακρίβεια από την μηδενική απόδοση. Αυτό υποδηλώνει ότι τα μοντέλα μαθαίνουν κάτι σημαντικό από τα δεδομένα.**

## **4.1 One-vs-One και One-vs-Rest**
---
Το target class αποτελείται από 3 διαφορετικές κλάσεις.
Συνεπώς:
1. Η one-vs-rest(ovr) υλοποίηση θα εκπαιδεύσει ένα SVM για κάθε ξεχωριστή κλάση, δηλαδή συνολικά 3 SVMs.
2. Η one-vs-one(ovo) υλοποίηση εκπαιδεύει στην γενικότερη περίπτωση n_classes * (n_classes - 1) / 2, δηλαδή στην περίπτωση μας πάλι 3 SVMs.

In [37]:
for kernel in ['linear', 'poly', 'rbf']:
    model = SVC(kernel=kernel,decision_function_shape='ovr')
    start = time()
    model.fit(X_train[:10000], Y_train[:10000])
    end = time()
    print("Number of support vectors = ", model.n_support_)
    print("Fit time: ", end-start)
    print("C = ", 1.0, "Train/Test acc = ", model.score(X_train[:10000], Y_train[:10000]), model.score(X_test, Y_test))

Number of support vectors =  [884 426 527]
Fit time:  2.142298936843872
C =  1.0 Train/Test acc =  0.9515 0.949275
Number of support vectors =  [3545 1484 2180]
Fit time:  3.2261271476745605
C =  1.0 Train/Test acc =  0.7087 0.70615
Number of support vectors =  [1957  598 1511]
Fit time:  2.0661098957061768
C =  1.0 Train/Test acc =  0.9258 0.919675


In [39]:
for kernel in ['linear', 'poly', 'rbf']:
    model = SVC(kernel=kernel,decision_function_shape='ovo')
    start = time()
    model.fit(X_train[:10000], Y_train[:10000])
    end = time()
    print("Number of support vectors = ", model.n_support_)
    print("Fit time: ", end-start)
    print("C = ", 1.0, "Train/Test acc = ", model.score(X_train[:10000], Y_train[:10000]), model.score(X_test, Y_test))

Number of support vectors =  [884 426 527]
Fit time:  2.078549861907959
C =  1.0 Train/Test acc =  0.9515 0.949275
Number of support vectors =  [3545 1484 2180]
Fit time:  3.224184274673462
C =  1.0 Train/Test acc =  0.7087 0.70615
Number of support vectors =  [1957  598 1511]
Fit time:  2.085172176361084
C =  1.0 Train/Test acc =  0.9258 0.919675


**Παρατηρούμε ότι οι αρχιτεκτονικές ovr και ovo παράγουν ακριβώς τα ίδια SVMs.**

**Αυτό συμβαίνει γιατί η sklearn βιβλιοθήκη υλοποίει εσωτερικά την στρατηγική ovo. Για στρατηγική ovr κατασκευάζεται ένας πίνακας από τον πίνακα ovo.
Παρατηρείται επίσης ότι η στρατηγική ovr αργεί ελάχιστα σε σχέση με την στρατηγική ovo.**

**Και οι δύο στρατηγικές παράγουν το ίδιο πλήθος SVMs.**

## **4.2 Σύγκριση με Nearest Centroid και k-Nearest Neighbors**
---
Παρακάτω γίνεται σύγκριση της απόδοσης και του χρόνου εκπαίδευσης με τους κατηγοριοποιητές Nearest Centroid και k-Nearest Neighbors.

Οι αποδόσεις των κατηγοριοποιητών Nearest Centroid και k-Nearest Neighbors από την προηγούμενη εργασία είναι:
1. **Nearest Centroid**: Accuracy=0.5420 Precision=0.5480 Recall=0.6290 F1-Score=0.5470
2. **k-NN με k=1**: Accuracy=0.8770 Precision=0.8710 Recall=0.8500 F1-Score=0.8600
3. **k-NN με k=3**: Accuracy=0.8780 Precision=0.8700 Recall=0.8550 F1-Score=0.8620

**Σχολιασμός**
---
Παρατηρούμε πως όλα τα SVMs ανεξαρτήτου πυρήνα τα πηγαίνουν καλύτερα σε θέμα απόδοσης από ότι οι κατηγοριοποιητές των Nearest Neighbors και Nearest Centroid. Τα SVMs επιτυγχάνουν μεγαλύτερο accuracy, precision, recall και f1-score κοντά στο 90%-96% ενώ οι Nearest Centroid φτάνουν μέχρι το 54% και οι Nearest Neighbors περίπου στο 87%. Παρόλα αυτά παρατηρείται ότι ο χρόνος εκπαίδευσης είναι σημαντικά μεγαλύτερος στα νευρωνικά δίκτυα.

Ο χρόνος εκπαίδευσης και πρόβλεψης των κατηγοριοποιητών Nearest Centroid και k-NN είναι οι εξής:
1. **Nearest Centroid**: Training Time:0.0119
2. **k-NN με k=1**: Training Time:0.0068 Predict Time: 2.449
3. **k-NN με k=3**: Training Time:0.0073 Predict Time: 2.4758

**Σχολιασμός**
---
Παρατηρούμε

## **4.3 Σύγκριση με Multilayered Perceptron**
---


In [40]:
# Import the necessary libraries
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers, models

In [37]:
Y_train_one_hot = to_categorical(Y_train, num_classes=3)    # One Hot encode the target train set
Y_test_one_hot = to_categorical(Y_test, num_classes=3)  # One Hot encode the target test set

In [44]:
# Initialize a good model from the previous project
mlp = models.Sequential()
# Input layer
mlp.add(layers.InputLayer(input_shape=(19,)))
mlp.add(layers.Dense(units=64, activation='relu'))
mlp.add(layers.Dense(units=32, activation='relu'))
mlp.add(layers.Dense(units=16, activation='relu'))

# Output layer
mlp.add(layers.Dense(units=3, activation='softmax'))

# Compile the model
mlp.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

# Train the model
mlp.fit(X_train, Y_train_one_hot, epochs=30, batch_size=32, validation_data=(X_test, Y_test_one_hot), verbose=0)

# **Κεφάλαιο 5: Συμπεράσματα**